In [122]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
import scipy.stats as stats
from geopy.geocoders import Nominatim
import time

In [159]:
geolocator = Nominatim(user_agent="grace")
location = geolocator.reverse("40.705356, -74.013438")
print(location.address)

GeocoderQuotaExceeded: HTTP Error 429: Too Many Requests

In [164]:
def address_from_coord(lat, long):
    start = time.time()
    geolocator = Nominatim(user_agent="jdmitchell")
    location = geolocator.reverse(f'{lat}, {long}')
    if time.time()-start<1:
        time.sleep(1.01-(time.time()-start))
    return location.address

In [165]:
def zipcode_from_coord(lat, long):
    return address_from_coord(lat, long).split(", ")[-2]

In [166]:
address_from_coord(40.84916900104506, -73.86835600032798)

GeocoderQuotaExceeded: HTTP Error 429: Too Many Requests

In [137]:
zipcode_from_coord(40.84916900104506, -73.86835600032798)

'10462'

In [13]:
df_subway = pd.read_csv("subway_entrances.csv")

In [82]:
df_subway['latlong']=df_subway['the_geom'].str[7:-1]

In [83]:
df_subway[df_subway.latlong == '-73.86835600032798 40.84916900104506']

OBJECTID                                URL  \
0      1734  http://web.mta.info/nyct/service/   

                                      NAME  \
0  Birchall Ave & Sagamore St at NW corner   

                                       the_geom LINE  \
0  POINT (-73.86835600032798 40.84916900104506)  2-5   

                                latlong                lat                long  
0  -73.86835600032798 40.84916900104506  40.84916900104506  -73.86835600032798

In [115]:
df_exerpt_subway = df_subway[:10]

In [121]:
df_exerpt_subway

OBJECTID                                URL  \
0      1734  http://web.mta.info/nyct/service/   
1      1735  http://web.mta.info/nyct/service/   
2      1736  http://web.mta.info/nyct/service/   
3      1737  http://web.mta.info/nyct/service/   
4      1738  http://web.mta.info/nyct/service/   
5      1739  http://web.mta.info/nyct/service/   
6      1740  http://web.mta.info/nyct/service/   
7      1741  http://web.mta.info/nyct/service/   
8      1742  http://web.mta.info/nyct/service/   
9      1743  http://web.mta.info/nyct/service/   

                                      NAME  \
0  Birchall Ave & Sagamore St at NW corner   
1  Birchall Ave & Sagamore St at NE corner   
2  Morris Park Ave & 180th St at NW corner   
3  Morris Park Ave & 180th St at NW corner   
4        Boston Rd & 178th St at SW corner   
5   Boston Rd & E Tremont Ave at NW corner   
6   Boston Rd & E Tremont Ave at NE corner   
7        Boston Rd & 178th St at SE corner   
8        Boston Rd & 178th St at NW corner   
9        Boston Rd & 174th St at SW corner   

                                        the_geom LINE  \
0   POINT (-73.86835600032798 40.84916900104506)  2-5   
1   POINT (-73.86821300022677 40.84912800131844)  2-5   
2   POINT (-73.87349900050798 40.84122300105249)  2-5   
3    POINT (-73.8728919997833 40.84145300067447)  2-5   
4   POINT (-73.87962300013866 40.84081500075867)  2-5   
5  POINT (-73.88000500027815 40.840434000875874)  2-5   
6   POINT (-73.87983300021861 40.84035400111976)  2-5   
7    POINT (-73.8795549998336 40.84063900116436)  2-5   
8   POINT (-73.87939700013239 40.84107800066419)  2-5   
9   POINT (-73.88804799985908 40.83732500129732)  2-5   

                                    latlong                 lat  \
0   [-73.86835600032798, 40.84916900104506]   40.84916900104506   
1   [-73.86821300022677, 40.84912800131844]   40.84912800131844   
2   [-73.87349900050798, 40.84122300105249]   40.84122300105249   
3    [-73.8728919997833, 40.84145300067447]   40.84145300067447   
4   [-73.87962300013866, 40.84081500075867]   40.84081500075867   
5  [-73.88000500027815, 40.840434000875874]  40.840434000875874   
6   [-73.87983300021861, 40.84035400111976]   40.84035400111976   
7    [-73.8795549998336, 40.84063900116436]   40.84063900116436   
8   [-73.87939700013239, 40.84107800066419]   40.84107800066419   
9   [-73.88804799985908, 40.83732500129732]   40.83732500129732   

                 long zipcode  
0  -73.86835600032798   10462  
1  -73.86821300022677   10462  
2  -73.87349900050798   10460  
3   -73.8728919997833   10460  
4  -73.87962300013866   10460  
5  -73.88000500027815   10460  
6  -73.87983300021861   10460  
7   -73.8795549998336   10460  
8  -73.87939700013239   10460  
9  -73.88804799985908   10460

In [119]:
df_exerpt_subway['zipcode'] = df_exerpt_subway.latlong.apply(lambda x: zipcode_from_coord(x[1], x[0]))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [85]:
df_subway['latlong'] = df_subway.latlong.str.split(' ')

In [86]:
df_subway['lat'] = df_subway['latlong'].apply(lambda x: x[1])

In [87]:
df_subway['long'] = df_subway['latlong'].apply(lambda x: x[0])

In [152]:
df_subway['zipcode'] = df_subway.iloc[800:1000].latlong.apply(lambda x: zipcode_from_coord(x[1], x[0]))
#make sure it's not deleting everything besides the section currently being updated

GeocoderQuotaExceeded: HTTP Error 429: Too Many Requests

In [158]:
df_subway.iloc[600:800]

OBJECTID                                URL  \
600       456  http://web.mta.info/nyct/service/   
601       457  http://web.mta.info/nyct/service/   
602       458  http://web.mta.info/nyct/service/   
603       459  http://web.mta.info/nyct/service/   
604       460  http://web.mta.info/nyct/service/   
605       461  http://web.mta.info/nyct/service/   
606       462  http://web.mta.info/nyct/service/   
607       463  http://web.mta.info/nyct/service/   
608       464  http://web.mta.info/nyct/service/   
609       465  http://web.mta.info/nyct/service/   
610       466  http://web.mta.info/nyct/service/   
611       467  http://web.mta.info/nyct/service/   
612       468  http://web.mta.info/nyct/service/   
613       469  http://web.mta.info/nyct/service/   
614       470  http://web.mta.info/nyct/service/   
615       471  http://web.mta.info/nyct/service/   
616       472  http://web.mta.info/nyct/service/   
617       473  http://web.mta.info/nyct/service/   
618       474  http://web.mta.info/nyct/service/   
619       475  http://web.mta.info/nyct/service/   
620       476  http://web.mta.info/nyct/service/   
621       477  http://web.mta.info/nyct/service/   
622       478  http://web.mta.info/nyct/service/   
623       479  http://web.mta.info/nyct/service/   
624       480  http://web.mta.info/nyct/service/   
625       481  http://web.mta.info/nyct/service/   
626       482  http://web.mta.info/nyct/service/   
627       483  http://web.mta.info/nyct/service/   
628       484  http://web.mta.info/nyct/service/   
629       485  http://web.mta.info/nyct/service/   
..        ...                                ...   
770       626  http://web.mta.info/nyct/service/   
771       627  http://web.mta.info/nyct/service/   
772       628  http://web.mta.info/nyct/service/   
773       629  http://web.mta.info/nyct/service/   
774       630  http://web.mta.info/nyct/service/   
775       631  http://web.mta.info/nyct/service/   
776       632  http://web.mta.info/nyct/service/   
777       633  http://web.mta.info/nyct/service/   
778       634  http://web.mta.info/nyct/service/   
779       635  http://web.mta.info/nyct/service/   
780       636  http://web.mta.info/nyct/service/   
781       637  http://web.mta.info/nyct/service/   
782       638  http://web.mta.info/nyct/service/   
783       639  http://web.mta.info/nyct/service/   
784       640  http://web.mta.info/nyct/service/   
785       641  http://web.mta.info/nyct/service/   
786       642  http://web.mta.info/nyct/service/   
787       643  http://web.mta.info/nyct/service/   
788       644  http://web.mta.info/nyct/service/   
789       645  http://web.mta.info/nyct/service/   
790       646  http://web.mta.info/nyct/service/   
791       647  http://web.mta.info/nyct/service/   
792       648  http://web.mta.info/nyct/service/   
793       649  http://web.mta.info/nyct/service/   
794       650  http://web.mta.info/nyct/service/   
795       651  http://web.mta.info/nyct/service/   
796       652  http://web.mta.info/nyct/service/   
797       653  http://web.mta.info/nyct/service/   
798       654  http://web.mta.info/nyct/service/   
799       655  http://web.mta.info/nyct/service/   

                                                  NAME  \
600            Grand Concourse & 179th St at NE corner   
601            Grand Concourse & 175th St at SW corner   
602            Grand Concourse & 175th St at NE corner   
603            Grand Concourse & 170th St at SW corner   
604            Grand Concourse & 170th St at NW corner   
605  Grand Concourse & 170th St at SE corner (exit ...   
606        Grand Concourse & McClellan St at SW corner   
607                 Walton Ave & 161st St at NW corner   
608                 Walton Ave & 161st St at NW corner   
609                 Walton Ave & 161st St at NW corner   
610                     6th Ave & 48th St at SW corner   
611                  6th Ave & 48-49th Sts at W corner   
612                  6th A